# 看穿你的意圖(pytorch版)

In [1]:
import codecs
import numpy as np
import random
import pickle
import os
import math

In [2]:
intent_data=codecs.open('intent_list.txt',mode='r',encoding='utf-8-sig').readlines()
intent_dict={}
for i in range(len(intent_data)):
    intent_dict[i]=intent_data[i].strip() 
intent_dict

{0: '打開app應用',
 1: '詢問公車路線',
 2: '數學計算',
 3: '閒聊',
 4: '詢問電影場次',
 5: '查詢聯絡人',
 6: '查詢食譜',
 7: '詢問日期',
 8: '電子郵件指令',
 9: '詢問電視節目',
 10: '詢問航班',
 11: '詢問健康資訊',
 12: '詢問樂透',
 13: '詢問地圖',
 14: '詢問足球賽事',
 15: '簡訊指令',
 16: '音樂指令',
 17: '詢問新聞',
 18: '詢問小說',
 19: '詢問詩詞',
 20: '詢問廣播節目',
 21: '詢問謎語',
 22: '鬧鐘時程設定',
 23: '詢問股票',
 24: '電話指令',
 25: '詢問火車班次',
 26: '詢問翻譯',
 27: '切換電視頻道',
 28: '影片播放指令',
 29: '詢問天氣',
 30: '開啟網站'}

In [3]:
train_data=codecs.open('../Data/ex13_train/Intent_TrainData.txt',mode='r',encoding='utf-8-sig').readlines()
maxlen=max([len(s) for s in train_data])
print("最大長度為:{0}".format(maxlen))
print(len(train_data))
print(train_data[:10])

最大長度為:52
2299
['打开熊猫看书\t0\r\n', 'opera浏览器\t0\r\n', '钛备份\t0\r\n', '打开sky电话\t0\r\n', '帮我找菜单\t0\r\n', '找愤怒的小鸟\t0\r\n', '正韵打开相机\t0\r\n', '我把相机打开\t0\r\n', '上QQ我\t0\r\n', '打开uc浏览器v八\t0\r\n']


In [4]:
test_data=codecs.open('../Data/ex13_train/Intent_TestData.txt',mode='r',encoding='utf-8-sig').readlines()
maxlen=max([len(s) for s in test_data])
print("最大長度為:{0}".format(maxlen))
print(len(test_data))
print(test_data[:10])

最大長度為:69
1437
['打开我的浏览器\t0\r\n', '打开qq浏览器\t0\r\n', '打开音乐播放器\t0\r\n', '启动浏览器\t0\r\n', '他开酷狗\t0\r\n', '帮我搜索三国杀\t0\r\n', '打开极品飞车\t0\r\n', '公交查询\t0\r\n', '打开搜狐新闻\t0\r\n', '图库\t0\r\n']


In [5]:
chars={}
with open('hanzi2vec256.pkl','rb') as fp:
    chars = pickle.load(fp)

In [6]:
random.shuffle(train_data) #把訓練數據集順序打亂
random.shuffle(test_data) #把訓練數據集順序打亂
idx0=0
idx1=0

def data_augumentation(txt):
    txt=list(txt)
    if random.randint(0,10)<=3 and len(txt)>6:#隨機對調字的順序
        pos= random.randint(1,len(txt)-2)
        w1=txt[pos]
        w2=txt[pos+1]
        txt[pos]=w2
        txt[pos+1]=w1
    if random.randint(0,100)%7<=3 and len(txt)>=6:
        wordlist=['的','了','若','可','然','也','或','，','。',' ','  ']
        w1=random.choice(wordlist)
        txt.insert(random.randint(1,len(txt)-2), w1)
    if random.randint(0,100)%5<2:
        wordlist=['，','。','？','！',' ','  ']
        w1=random.choice(wordlist)
        txt.append(w1)
    return ''.join(txt)

def get_next_minibatch(minibatch_size=16,is_train=True):
    global idx0,idx1
    
    features = []
    labels = []
    groundtruths = []
    while len(features) < minibatch_size:
        try:
            items=[]
            if is_train:
                items=train_data[idx0].strip().split('\t')
            else:
                items=test_data[idx1].strip().split('\t')
            txt=items[0]
            new_txt=txt
            if is_train:
                new_txt=data_augumentation(txt)
            lab=int(items[1])
            feature=np.zeros((256,128)) #預設都為零

            for i in range(128):
                if i< len(new_txt):
                    feature[:,i]=chars[new_txt[i].lower()] if new_txt[i].lower() in chars else np.random.uniform(-1,1,256)
                    #替換為字向量，若是沒有這個字，填隨機亂數
            features.append(feature)
            labels.append(lab)
            groundtruths.append(list(txt))
            if is_train:
                idx0 += 1
                if idx0 >= len(train_data):
                    idx0 = 0
                    random.shuffle(train_data)
            else:
                idx1 += 1
                if idx1 >= len(test_data):
                    idx1 = 0
                    random.shuffle(test_data)
        except Exception as e:
            print(e)
            
    return np.asarray(features).astype(np.float32), np.asarray(labels).astype(np.int64), groundtruths


features,labels,groundtruths=get_next_minibatch(minibatch_size=3)
print(features)
print(labels)
print(groundtruths)


[[[ 0.44852316 -0.1533463  -0.517445   ...  0.          0.
    0.        ]
  [-0.43695968  0.1154801  -0.1230142  ...  0.          0.
    0.        ]
  [-0.14062339 -0.46853998 -0.04405463 ...  0.          0.
    0.        ]
  ...
  [ 0.91536975 -0.15886183  1.0464317  ...  0.          0.
    0.        ]
  [-0.30758566 -0.27976     0.330975   ...  0.          0.
    0.        ]
  [-0.56287533  0.26237872  0.14345555 ...  0.          0.
    0.        ]]

 [[-0.04448497 -0.2417378   0.09759909 ...  0.          0.
    0.        ]
  [ 0.17824216  0.47825775 -0.09346677 ...  0.          0.
    0.        ]
  [ 0.48564908  0.12558667 -0.03327355 ...  0.          0.
    0.        ]
  ...
  [ 1.134174    0.35537347  0.6736903  ...  0.          0.
    0.        ]
  [ 0.06045923 -0.06723896  0.2577471  ...  0.          0.
    0.        ]
  [ 0.07368705 -0.77622026 -1.2732855  ...  0.          0.
    0.        ]]

 [[-0.06707906 -1.3728026  -0.983686   ...  0.          0.
    0.        ]
  [ 0.402

In [7]:
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd

# 是否使用GPU
is_cuda = torch.cuda.is_available()

![md_images](../Images/textcnn.jpg)

In [8]:
class TextCNN(nn.Module):
    def __init__(self, drop_prob=0.5, embedding_dim=256,n_classes=31):
        super(TextCNN, self).__init__()
        self.dropout = nn.Dropout(drop_prob)
        self.conv3 = nn.Sequential(
            nn.Conv1d(in_channels=embedding_dim, out_channels=64, kernel_size=3,stride=2,  padding=1),
            nn.BatchNorm1d(64),
            nn.Tanh())

        self.conv5 = nn.Sequential(
            nn.Conv1d(in_channels=embedding_dim, out_channels=64, kernel_size=5, stride=2,padding=2),
            nn.BatchNorm1d(64),
            nn.Tanh())

        self.conv7 = nn.Sequential(
            nn.Conv1d(in_channels=embedding_dim, out_channels=64, kernel_size=7,stride=2, padding=3),
            nn.BatchNorm1d(64),
            nn.Tanh())

        self.conv1d3 = nn.Conv1d(in_channels=64*3, out_channels=64, kernel_size=3, padding=1)
        self.conv1d1 = nn.Conv1d(in_channels=64, out_channels=16, kernel_size=1)
        self.fc = nn.Linear(16*64, n_classes)

    def forward(self, x):
        x=self.dropout(x)
        x3 = self.conv3(x)
        x5 = self.conv5(x)
        x7 = self.conv7(x)

        x = torch.cat((x3, x5, x7),  dim=1)  
        x = self.dropout(x)
        x = self.conv1d3(x)
        x = self.conv1d1(x)
        x = x.view(x.size(0), -1) 
        logits = self.fc(x)
        return F.softmax(logits)
    
#為了解決類別間不均衡的焦距損失
class FocalLoss(nn.Module):
    def __init__(self, gamma=0, alpha=None, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha,(float)): self.alpha = t.Tensor([alpha,1-alpha])
        if isinstance(alpha,list): self.alpha = t.Tensor(alpha)
        self.size_average = size_average

    def forward(self, input, target):
        if input.dim()>2:
            input = input.view(input.size(0),input.size(1),-1)  # N,C,H,W => N,C,H*W
            input = input.transpose(1,2)    # N,C,H*W => N,H*W,C
            input = input.contiguous().view(-1,input.size(2))   # N,H*W,C => N*H*W,C
        target = target.view(-1,1)

        logpt = F.log_softmax(input)
        logpt = logpt.gather(1,target)
        logpt = logpt.view(-1)
        pt = Variable(logpt.data.exp())

        if self.alpha is not None:
            if self.alpha.type()!=input.data.type():
                self.alpha = self.alpha.type_as(input.data)
            at = self.alpha.gather(0,target.data.view(-1))
            logpt = logpt * Variable(at)

        loss = -1 * (1-pt)**self.gamma * logpt
        if self.size_average: return loss.mean()
        else: return loss.sum()

In [ ]:
model = TextCNN()
if os.path.exists('Models/TextCNN_pytorch.cnn'):
    model=torch.load('Models/TextCNN_pytorch.cnn')
    print('recovered!!')

model.train()
model_optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)


if is_cuda :
    model.cuda()


recovered!!


In [ ]:

epochs=300
minibatch_size=32
print('start epoch!')
for epoch in range(epochs):
    mbs=0
    while mbs<1000:
        features, labels, truths = get_next_minibatch(minibatch_size)
        inputs, targets = torch.from_numpy(features), torch.from_numpy(labels.astype(np.int64))     
        inputs, targets = Variable(inputs), Variable(targets)
        if is_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        model.zero_grad()
        output = model.forward(inputs)
        
        loss = nn.CrossEntropyLoss()(output, targets)+FocalLoss(gamma=2)(output, targets)
        loss.backward()
        err=np.mean(np.not_equal(np.argmax(output.cpu().detach().numpy(),-1).astype(np.int64),labels.astype(np.int64)))
        model_optimizer.step()
        if mbs % 10 == 0:
            #列印訓練狀態
            print("Epoch: {}/{}...".format(epoch+1, epochs),
                        "Step: {}...".format(mbs),
                        "Loss: {:.4f}...".format(loss.data.item()),
                        "Accuracy: {:.3%}...".format(1-err))
        if mbs % 20 == 0:
            torch.save(model, 'Models/TextCNN_pytorch{0}.cnn'.format(epoch))
            torch.save(model, 'Models/TextCNN_pytorch.cnn')
        if mbs%100==0 and mbs>0 :
            test_features, test_labels, test_truths = get_next_minibatch(minibatch_size,is_train=False)
            test_inputs, test_targets = torch.from_numpy(test_features), torch.from_numpy(test_labels.astype(np.int64))     
            test_inputs, test_targets = Variable(test_inputs), Variable(test_targets)
            if is_cuda:
                test_inputs, test_targets = test_inputs.cuda(), test_targets.cuda()
            pred=model.forward(test_inputs).cpu().detach().numpy()
            for i in range(5):
                sentence=test_truths[i]
                if len(sentence)>=3:
                    result=np.argmax(pred[i])
                    label=test_labels[i]
                    print('{0}  ({1})=>({2})'.format(''.join(sentence),intent_dict[label],intent_dict[result]))
        mbs += 1

start epoch!


C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:36: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:55: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
C:\Anaconda3\lib\site-packages\torch\serialization.py:250: UserWarning: Couldn't retrieve source code for container of type TextCNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch: 1/300... Step: 0... Loss: 5.9525... Accuracy: 37.500%...
Epoch: 1/300... Step: 10... Loss: 5.9475... Accuracy: 37.500%...
Epoch: 1/300... Step: 20... Loss: 5.9275... Accuracy: 37.500%...
Epoch: 1/300... Step: 30... Loss: 5.9421... Accuracy: 37.500%...
Epoch: 1/300... Step: 40... Loss: 5.9600... Accuracy: 37.500%...
Epoch: 1/300... Step: 50... Loss: 5.8770... Accuracy: 40.625%...
Epoch: 1/300... Step: 60... Loss: 5.8792... Accuracy: 40.625%...
Epoch: 1/300... Step: 70... Loss: 6.0267... Accuracy: 34.375%...
Epoch: 1/300... Step: 80... Loss: 6.1526... Accuracy: 28.125%...
Epoch: 1/300... Step: 90... Loss: 5.7655... Accuracy: 46.875%...
Epoch: 1/300... Step: 100... Loss: 5.7410... Accuracy: 46.875%...
翻译我是一只狗  (詢問翻譯)=>(閒聊)
最近下午有什么电影？  (詢問電視節目)=>(影片播放指令)
我想订8月15晚上北京飞合肥的机票  (詢問航班)=>(詢問電視節目)
郑州天气  (詢問天氣)=>(閒聊)
合肥的天气  (詢問天氣)=>(閒聊)
Epoch: 1/300... Step: 110... Loss: 5.6702... Accuracy: 50.000%...
Epoch: 1/300... Step: 120... Loss: 6.2862... Accuracy: 21.875%...
Epoch: 1/300... Step: 130